In [1]:
# Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ast
import cv2
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

# Import Keras
from matplotlib import pyplot 
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
# Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train = pd.read_csv('df_train_with_new_test.csv')
target_train = train['is_iceberg']
test = pd.read_json('data/processed/test.json')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce') # We have only 133 NAs.
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
x_angle = train['inc_angle']
x_test_angle = test['inc_angle']

x_train = np.load("x_train.npy")
x_test = np.load('x_test.npy')

In [3]:
batch_size = 64
# Define the image transformations here

gen = ImageDataGenerator(width_shift_range = 0.1,
                        height_shift_range = 0.1,
                        zoom_range = 0.2,
                        rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and the angle arrays

def gen_flow_for_two_inputs(x1, x2, y):
    genx1 = gen.flow(x1, y, batch_size=batch_size, seed=55)
    genx2 = gen.flow(x1, x2, batch_size=batch_size, seed=55)
    while True:
        x1i = genx1.next()
        x2i = genx2.next()
        # Assert arrays are equal - this was for peace of mind but slows down training.
        yield [x1i[0], x2i[1]], x1i[1]

In [4]:
# Finally create generator
def get_callbacks(filepath, patience=40):
    es = EarlyStopping('val_loss', patience=40, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=0.00001)
    
    return [es, msave, reduce_lr]



def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1,)(input_2)
    base_model = VGG19(weights='imagenet', include_top = False, input_shape=x_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    for layer in model.layers[:3]:
        layer.trainable = False
    for layer in model.layers[3:]:
        layer.trainable = True
    optimizer = Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [5]:
# Using K-fold cross validation with Data Augmentation.

def myAngleCV(x_train, x_angle, x_test):
    k = 4
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=16).split(x_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0 * target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print("===========FOLD============")
        x_train_cv = x_train[train_idx]
        y_train_cv = target_train[train_idx]
        x_holdout = x_train[test_idx]
        y_holdout = target_train[test_idx]
        
        # Angle
        x_angle_cv = x_angle[train_idx]
        x_angle_hold = x_angle[test_idx]
        
        # Define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=40)
        gen_flow = gen_flow_for_two_inputs(x_train_cv, x_angle_cv, y_train_cv)
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(gen_flow,
                                 steps_per_epoch=24,
                                 epochs=150,
                                 shuffle=True,
                                 verbose=1,
                                 validation_data=([x_holdout, x_angle_hold], y_holdout),
                                 callbacks=callbacks)
        
        # Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        # Getting Training Score
        score = galaxyModel.evaluate([x_train_cv, x_angle_cv], y_train_cv, verbose=0)
        print("Train loss:", score[0])
        print("Train accuracy:", score[1])
        # Getting Test Score
        score = galaxyModel.evaluate([x_holdout, x_angle_hold], y_holdout, verbose=0)
        print("Test Loss:", score[0])
        print("Test accuracy:", score[1])
        
        # Getting validation score.
        pred_valid = galaxyModel.predict([x_holdout, x_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        # Getting Test Scores
        temp_test = galaxyModel.predict([x_test, x_test_angle])
        y_test_pred_log += temp_test.reshape(temp_test.shape[0])
        
        # Getting Train Scores
        temp_train = galaxyModel.predict([x_train, x_angle])
        y_train_pred_log += temp_train.reshape(temp_train.shape[0])
        
    y_test_pred_log = y_test_pred_log/k
    y_train_pred_log = y_train_pred_log/k
    
    print("Train log loss validation= ", log_loss(target_train, y_train_pred_log))
    print("Test log loss validation= ", log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [6]:
preds = myAngleCV(x_train, x_angle, x_test)

===========FOLD============


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/150
24/24 [==============================] - 10s - loss: 0.6395 - acc: 0.6966 - val_loss: 0.3100 - val_acc: 0.8767
Epoch 2/150
24/24 [==============================] - 6s - loss: 0.3367 - acc: 0.8796 - val_loss: 0.1654 - val_acc: 0.9424
Epoch 3/150
24/24 [==============================] - 6s - loss: 0.1879 - acc: 0.9358 - val_loss: 0.1551 - val_acc: 0.9448
Epoch 4/150
24/24 [==============================] - 6s - loss: 0.2149 - acc: 0.9232 - val_loss: 0.1110 - val_acc: 0.9562
Epoch 5/150
24/24 [==============================] - 6s - loss: 0.1572 - acc: 0.9438 - val_loss: 0.0794 - val_acc: 0.9627
Epoch 6/150
24/24 [==============================] - 5s - loss: 0.1268 - acc: 0.9492 - val_loss: 0.0829 - val_acc: 0.9570
Epoch 7/150
24/24 [==============================] - 5s - loss: 0.1109 - acc: 0.9538 - val_loss: 0.0851 - val_acc: 0.9651
Epoch 8/150
24/24 [==============================] - 5s - loss: 0.1162 - acc: 0.9599 - val_loss: 0.1064 - val_acc: 0.9578
Epoch 9/150
24/24 [====

24/24 [==============================] - 6s - loss: 0.0382 - acc: 0.9883 - val_loss: 0.1150 - val_acc: 0.9676
Epoch 68/150
24/24 [==============================] - 5s - loss: 0.0202 - acc: 0.9939 - val_loss: 0.0936 - val_acc: 0.9700
Epoch 69/150
24/24 [==============================] - 5s - loss: 0.0170 - acc: 0.9941 - val_loss: 0.1119 - val_acc: 0.9716
Epoch 70/150
24/24 [==============================] - 6s - loss: 0.0169 - acc: 0.9935 - val_loss: 0.1102 - val_acc: 0.9692
Train loss: 0.0478131136836
Train accuracy: 0.983766233766
Test Loss: 0.0640876846047
Test accuracy: 0.975669099757
===========FOLD============
Epoch 1/150
24/24 [==============================] - 8s - loss: 0.9355 - acc: 0.6035 - val_loss: 0.4298 - val_acc: 0.6456
Epoch 2/150
24/24 [==============================] - 6s - loss: 0.4092 - acc: 0.8509 - val_loss: 0.2779 - val_acc: 0.8938
Epoch 3/150
24/24 [==============================] - 6s - loss: 0.3391 - acc: 0.8657 - val_loss: 0.1762 - val_acc: 0.9343
Epoch 4/150

24/24 [==============================] - 5s - loss: 0.0296 - acc: 0.9909 - val_loss: 0.1546 - val_acc: 0.9676
Epoch 63/150
24/24 [==============================] - 5s - loss: 0.0270 - acc: 0.9903 - val_loss: 0.1435 - val_acc: 0.9627
Epoch 64/150
24/24 [==============================] - 5s - loss: 0.0159 - acc: 0.9948 - val_loss: 0.1256 - val_acc: 0.9667
Epoch 65/150
24/24 [==============================] - 5s - loss: 0.0294 - acc: 0.9889 - val_loss: 0.1262 - val_acc: 0.9659
Epoch 66/150
24/24 [==============================] - 5s - loss: 0.0280 - acc: 0.9883 - val_loss: 0.1313 - val_acc: 0.9659
Epoch 67/150
24/24 [==============================] - 5s - loss: 0.0215 - acc: 0.9922 - val_loss: 0.1210 - val_acc: 0.9676
Epoch 68/150
24/24 [==============================] - 5s - loss: 0.0173 - acc: 0.9928 - val_loss: 0.1480 - val_acc: 0.9651
Epoch 69/150
24/24 [==============================] - 5s - loss: 0.0306 - acc: 0.9909 - val_loss: 0.1303 - val_acc: 0.9676
Epoch 70/150
24/24 [=========

24/24 [==============================] - 6s - loss: 0.0429 - acc: 0.9818 - val_loss: 0.0982 - val_acc: 0.9635
Epoch 58/150
24/24 [==============================] - 6s - loss: 0.0428 - acc: 0.9837 - val_loss: 0.0926 - val_acc: 0.9651
Epoch 59/150
24/24 [==============================] - 5s - loss: 0.0315 - acc: 0.9870 - val_loss: 0.0950 - val_acc: 0.9659
Epoch 60/150
24/24 [==============================] - 5s - loss: 0.0436 - acc: 0.9805 - val_loss: 0.0979 - val_acc: 0.9651
Epoch 61/150
24/24 [==============================] - 5s - loss: 0.0459 - acc: 0.9848 - val_loss: 0.0943 - val_acc: 0.9675
Epoch 62/150
24/24 [==============================] - 6s - loss: 0.0361 - acc: 0.9870 - val_loss: 0.0964 - val_acc: 0.9692
Epoch 63/150
24/24 [==============================] - 5s - loss: 0.0295 - acc: 0.9881 - val_loss: 0.1043 - val_acc: 0.9643
Epoch 64/150
24/24 [==============================] - 5s - loss: 0.0454 - acc: 0.9837 - val_loss: 0.0886 - val_acc: 0.9692
Epoch 65/150
24/24 [=========

24/24 [==============================] - 5s - loss: 0.0357 - acc: 0.9902 - val_loss: 0.1041 - val_acc: 0.9683
Epoch 49/150
24/24 [==============================] - 5s - loss: 0.0374 - acc: 0.9829 - val_loss: 0.1016 - val_acc: 0.9708
Epoch 50/150
24/24 [==============================] - 5s - loss: 0.0328 - acc: 0.9857 - val_loss: 0.1323 - val_acc: 0.9716
Epoch 51/150
24/24 [==============================] - 5s - loss: 0.0275 - acc: 0.9883 - val_loss: 0.1289 - val_acc: 0.9626
Epoch 52/150
24/24 [==============================] - 5s - loss: 0.0422 - acc: 0.9837 - val_loss: 0.1243 - val_acc: 0.9675
Epoch 53/150
24/24 [==============================] - 5s - loss: 0.0222 - acc: 0.9889 - val_loss: 0.1193 - val_acc: 0.9675
Epoch 54/150
24/24 [==============================] - 5s - loss: 0.0241 - acc: 0.9914 - val_loss: 0.1151 - val_acc: 0.9732
Epoch 55/150
24/24 [==============================] - 5s - loss: 0.0326 - acc: 0.9863 - val_loss: 0.0983 - val_acc: 0.9708
Epoch 56/150
24/24 [=========

In [9]:
# Submission for each day
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
submission.to_csv('vgg19-test-data-final.csv', index=False)